<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 25


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс SavingsAccount в C#, который будет представлять
специализированные учетные записи для сбережений в банке. На основе этого
класса разработать 2-3 производных класса, демонстрирующих принципы
наследования и полиморфизма. В каждом из классов должны быть реализованы
новые атрибуты и методы, а также переопределены некоторые методы базового
класса для демонстрации полиморфизма.

Требования к базовому классу SavingsAccount:

• Атрибуты: ID счета (AccountId), Баланс (Balance), Процентная ставка
(InterestRate).

• Методы:

o Deposit(): метод для внесения денег на счет.

o Withdraw(): метод для снятия денег со счета.

o CalculateInterest(): метод для расчета процентов по счету.
Требования к производным классам:
1. СтуденческаяУчетнаяЗапись (StudentAccount): Должна содержать
дополнительные атрибуты, такие как Год обучения (YearOfStudy).
Метод CalculateInterest() должен быть переопределен для применения
сниженной процентной ставки для студентов.
2. ПремиумУчетнаяЗапись (PremiumAccount): Должна содержать
дополнительные атрибуты, такие как Минимальный баланс
(MinimumBalance). Метод Withdraw() должен быть переопределен для
ограничения снятия средств до минимального баланса.
3. ИнвестиционныйУчет (InvestmentAccount) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Инвестиционный
портфель (PortfolioValue). Метод Deposit() должен быть переопределен для
автоматического инвестирования части внесенных средств.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface INotificationService
{
    void SendNotification(string message, string accountId);
}

public interface IAuditService
{
    void LogTransaction(string accountId, string transactionType, decimal amount, DateTime timestamp);
}

public interface ITransactionFee
{
    decimal TransactionFee { get; }
    void ApplyTransactionFee(decimal amount);
}

public interface ILoyaltyProgram
{
    int LoyaltyPoints { get; }
    void AddLoyaltyPoints(int points);
    void RedeemLoyaltyPoints();
}

public class EmailNotificationService : INotificationService
{
    public void SendNotification(string message, string accountId)
    {
        Console.WriteLine($"Email отправлен для счета {accountId}: {message}");
    }
}

public class FileAuditService : IAuditService
{
    public void LogTransaction(string accountId, string transactionType, decimal amount, DateTime timestamp)
    {
        Console.WriteLine($"Аудит: Счет {accountId}, Тип: {transactionType}, Сумма: {amount:C}, Время: {timestamp}");
    }
}
public delegate void DisplayDelegate();
public class SavingAccount
{
    public string AccountId { get; protected set; }
    public decimal Balance { get; protected set; }
    public decimal InterestRate { get; protected set; }
    public DateTime OpeningDate { get; protected set; } 
    public string OwnerName { get; protected set; }
    public bool IsActive { get; protected set; }
    
    public string Currency { get; protected set; }
    public DateTime LastTransactionDate { get; protected set; }
    public int TransactionCount { get; protected set; }
    public decimal DailyWithdrawalLimit { get; protected set; }
    protected decimal DailyWithdrawalAmount { get; set; }
    private DateTime LastWithdrawalReset { get; set; }

    protected readonly INotificationService _notificationService;
    protected readonly IAuditService _auditService;

    public delegate void AccountHandler(string message);
    public event AccountHandler AccountUpdated;
    public void Purchase(decimal newBalance, decimal purchasessum)
    {
        if (Balance >= purchasessum)
        {
            newBalance = Balance - purchasessum;
            AccountUpdated?.Invoke($"На счете осталось: {newBalance}");
        }
        else
        {
            AccountUpdated?.Invoke($"На счете недостаточно средств.");
        }
    }
    public SavingAccount(string accountId, decimal balance, decimal interestRate, string ownerName, 
                        INotificationService notificationService, IAuditService auditService)
    {
        AccountId = accountId;
        Balance = balance;
        InterestRate = interestRate;
        OwnerName = ownerName;
        OpeningDate = DateTime.Now;
        IsActive = true;
        Currency = "RUB";
        LastTransactionDate = DateTime.Now;
        TransactionCount = 0;
        DailyWithdrawalLimit = 50000;
        DailyWithdrawalAmount = 0;
        LastWithdrawalReset = DateTime.Now;
        
        _notificationService = notificationService;
        _auditService = auditService;
    }

    protected virtual void UpdateTransactionData()
    {
        LastTransactionDate = DateTime.Now;
        TransactionCount++;
        
        if ((DateTime.Now - LastWithdrawalReset).Days >= 1)
        {
            DailyWithdrawalAmount = 0;
            LastWithdrawalReset = DateTime.Now;
        }
    }

    protected virtual bool CheckDailyWithdrawalLimit(decimal amount)
    {
        return DailyWithdrawalAmount + amount <= DailyWithdrawalLimit;
    }

    public virtual void ChangeCurrency(string newCurrency)
    {
        string oldCurrency = Currency;
        Currency = newCurrency;
        _notificationService.SendNotification($"Валюта счета изменена с {oldCurrency} на {newCurrency}", AccountId);
    }

    public virtual void ResetDailyWithdrawal()
    {
        DailyWithdrawalAmount = 0;
        Console.WriteLine("Дневной лимит снятия сброшен");
    }

    public virtual decimal GetAvailableBalance()
    {
        return Balance;
    }

    public virtual void FreezeAccount()
    {
        IsActive = false;
        _notificationService.SendNotification("Счет заморожен", AccountId);
    }

    public virtual void UnfreezeAccount()
    {
        IsActive = true;
        _notificationService.SendNotification("Счет разморожен", AccountId);
    }
    public virtual void DisplayInfo()
    {
        Console.WriteLine($"ID: {AccountId}; Balance: {Balance}; IsActive = {IsActive}");
    }

    public virtual void Deposit(decimal amount)
    {
        if (amount <= 0)
        {
            throw new ArgumentException("Сумма должна быть положительной");
        }
        if (!IsActive)
        {
            throw new InvalidOperationException("Счет не активен");
        }

        Balance += amount;
        UpdateTransactionData();
        _auditService.LogTransaction(AccountId, "DEPOSIT", amount, DateTime.Now);
        _notificationService.SendNotification($"Внесено на счет {amount:C}. Текущий баланс {Balance:C}", AccountId);
        
        Console.WriteLine($"Внесено на счет {amount:C}. Текущий баланс {Balance:C}");
    }

    public virtual void Deposit(decimal amount, string description)
    {
        Deposit(amount);
        Console.WriteLine($"Описание: {description}");
    }

    public virtual bool Withdraw(decimal amount)
    {
        if (amount <= 0)
        {
            throw new ArgumentException("Сумма должна быть положительной");
        }
        if (!IsActive)
        {
            Console.WriteLine("Счет не активен");
            return false;
        }
        if (!CheckDailyWithdrawalLimit(amount))
        {
            Console.WriteLine($"Превышен дневной лимит снятия. Доступно: {DailyWithdrawalLimit - DailyWithdrawalAmount:C}");
            return false;
        }

        if (amount > Balance)
        {
            Console.WriteLine("Недостаточно средств на счете");
            return false;
        }
        else
        {
            Balance -= amount;
            DailyWithdrawalAmount += amount;
            UpdateTransactionData();
            _auditService.LogTransaction(AccountId, "WITHDRAWAL", amount, DateTime.Now);
            _notificationService.SendNotification($"Снято {amount:C}. Текущий баланс {Balance:C}", AccountId);
            
            Console.WriteLine($"Снято {amount:C}. Текущий баланс {Balance:C}");
            return true;
        }
    }

    public virtual bool Withdraw(decimal amount, string purpose)
    {
        bool result = Withdraw(amount);
        if (result)
        {
            Console.WriteLine($"Цель снятия: {purpose}");
        }
        return result;
    }

    public virtual decimal CalculateInterest()
    {
        decimal interest = Balance * InterestRate / 100;
        Console.WriteLine($"Начисленные проценты {interest:C}");
        return interest;
    }

    public virtual decimal CalculateInterest(int months)
    {
        decimal interest = Balance * InterestRate / 100 * months;
        Console.WriteLine($"Начисленные проценты за {months} месяцев: {interest:C}");
        return interest;
    }

    public virtual void AccountInfo()
    {
        Console.WriteLine($"Счет: {AccountId}, Владелец: {OwnerName}, Баланс: {Balance:C}, Ставка: {InterestRate}%");
    }

    public virtual void AccountInfo(bool detailed)
    {
        AccountInfo();
        if (detailed)
        {
            Console.WriteLine($"Дата открытия: {OpeningDate}, Активен: {IsActive}, Возраст счета: {GetAccountAgeInDays()} дней");
            Console.WriteLine($"Валюта: {Currency}, Транзакций: {TransactionCount}, Последняя операция: {LastTransactionDate}");
            Console.WriteLine($"Дневной лимит: {DailyWithdrawalLimit:C}, Использовано: {DailyWithdrawalAmount:C}");
        }
    }

    public virtual void ApplyInterest()
    {
        decimal interest = CalculateInterest();
        Balance += interest;
        _auditService.LogTransaction(AccountId, "INTEREST", interest, DateTime.Now);
        Console.WriteLine($"Проценты {interest:C} начислены на счет. Новый баланс: {Balance:C}");
    }

    public virtual void CloseAccount()
    {
        if (Balance > 0)
        {
            Console.WriteLine($"Нельзя закрыть счет с положительным балансом {Balance:C}");
            return;
        }
        IsActive = false;
        _notificationService.SendNotification("Счет закрыт", AccountId);
        Console.WriteLine($"Счет {AccountId} закрыт");
    }

    public int GetAccountAgeInDays()
    {
        return (DateTime.Now - OpeningDate).Days;
    }
}

public class AdvancedBusinessAccount : SavingAccount, ITransactionFee, ILoyaltyProgram
{
    public string CompanyName { get; private set; }
    public string BusinessType { get; private set; }
    public decimal TransactionFee { get; private set; }
    private int _loyaltyPoints;
    public int LoyaltyPoints => _loyaltyPoints;

    public string TaxId { get; private set; }
    public decimal MonthlyFee { get; private set; }
    public DateTime LastFeeDate { get; private set; }
    public bool HasOverdraft { get; private set; }
    public decimal OverdraftLimit { get; private set; }

    decimal ITransactionFee.TransactionFee => TransactionFee;
    int ILoyaltyProgram.LoyaltyPoints => _loyaltyPoints;

    public AdvancedBusinessAccount(string accountId, decimal balance, decimal interestRate, string ownerName, 
                                 string companyName, string businessType, string taxId,
                                 INotificationService notificationService, IAuditService auditService)
        : base(accountId, balance, interestRate, ownerName, notificationService, auditService)
    {
        CompanyName = companyName;
        BusinessType = businessType;
        TaxId = taxId;
        TransactionFee = 0.5m;
        _loyaltyPoints = 0;
        MonthlyFee = 100m;
        LastFeeDate = DateTime.Now;
        HasOverdraft = true;
        OverdraftLimit = 5000m;
        DailyWithdrawalLimit = 100000m;
    }

    public void ApplyMonthlyFee()
    {
        if ((DateTime.Now - LastFeeDate).Days >= 30)
        {
            if (Balance >= MonthlyFee)
            {
                Balance -= MonthlyFee;
                LastFeeDate = DateTime.Now;
                _auditService.LogTransaction(AccountId, "MONTHLY_FEE", MonthlyFee, DateTime.Now);
                Console.WriteLine($"Списана ежемесячная комиссия: {MonthlyFee:C}");
            }
            else
            {
                Console.WriteLine("Недостаточно средств для списания ежемесячной комиссии");
            }
        }
    }

    public void SetOverdraftLimit(decimal limit)
    {
        OverdraftLimit = limit;
        _notificationService.SendNotification($"Лимит овердрафта установлен: {limit:C}", AccountId);
    }

    public void ProcessTaxDeduction(decimal amount)
    {
        if (amount <= Balance)
        {
            Balance -= amount;
            _auditService.LogTransaction(AccountId, "TAX_DEDUCTION", amount, DateTime.Now);
            Console.WriteLine($"Налоговый вычет {amount:C} обработан");
        }
    }

    protected override bool CheckDailyWithdrawalLimit(decimal amount)
    {
        return DailyWithdrawalAmount + amount <= DailyWithdrawalLimit;
    }

    public override decimal GetAvailableBalance()
    {
        return Balance + (HasOverdraft ? OverdraftLimit : 0);
    }

    public override bool Withdraw(decimal amount)
    {
        if (amount <= 0)
        {
            throw new ArgumentException("Сумма должна быть положительной");
        }
        if (!IsActive)
        {
            Console.WriteLine("Счет не активен");
            return false;
        }
        if (!CheckDailyWithdrawalLimit(amount))
        {
            Console.WriteLine($"Превышен дневной лимит снятия. Доступно: {DailyWithdrawalLimit - DailyWithdrawalAmount:C}");
            return false;
        }

        decimal availableBalance = GetAvailableBalance();
        if (amount > availableBalance)
        {
            Console.WriteLine($"Недостаточно средств. Доступно: {availableBalance:C}");
            return false;
        }

        Balance -= amount;
        DailyWithdrawalAmount += amount;
        UpdateTransactionData();
        _auditService.LogTransaction(AccountId, "WITHDRAWAL", amount, DateTime.Now);
        
        ((ITransactionFee)this).ApplyTransactionFee(amount);
        
        _notificationService.SendNotification($"Снято {amount:C}. Текущий баланс {Balance:C}", AccountId);
        
        Console.WriteLine($"Снято {amount:C}. Текущий баланс {Balance:C}");
        return true;
    }

    void ITransactionFee.ApplyTransactionFee(decimal amount)
    {
        decimal fee = amount * TransactionFee / 100;
        Balance -= fee;
        _auditService.LogTransaction(AccountId, "TRANSACTION_FEE", fee, DateTime.Now);
        Console.WriteLine($"Применена комиссия за транзакцию: {fee:C}");
    }

    void ILoyaltyProgram.AddLoyaltyPoints(int points)
    {
        _loyaltyPoints += points;
        Console.WriteLine($"Добавлено {points} баллов лояльности. Всего: {_loyaltyPoints}");
    }

    void ILoyaltyProgram.RedeemLoyaltyPoints()
    {
        if (_loyaltyPoints >= 100)
        {
            decimal bonus = _loyaltyPoints * 0.1m;
            Balance += bonus;
            Console.WriteLine($"Обменено {_loyaltyPoints} баллов на {bonus:C}");
            _loyaltyPoints = 0;
        }
        else
        {
            Console.WriteLine($"Недостаточно баллов для обмена. Требуется: 100, доступно: {_loyaltyPoints}");
        }
    }

    public override void Deposit(decimal amount)
    {
        if (amount <= 0)
        {
            throw new ArgumentException("Сумма должна быть положительной");
        }
        if (!IsActive)
        {
            throw new InvalidOperationException("Счет не активен");
        }

        Balance += amount;
        UpdateTransactionData();
        _auditService.LogTransaction(AccountId, "DEPOSIT", amount, DateTime.Now);
        _notificationService.SendNotification($"Внесено на счет {amount:C}. Текущий баланс {Balance:C}", AccountId);
        
        Console.WriteLine($"Внесено на счет {amount:C}. Текущий баланс {Balance:C}");
        
        ((ITransactionFee)this).ApplyTransactionFee(amount);
        ((ILoyaltyProgram)this).AddLoyaltyPoints((int)(amount / 10));
    }

    public override void AccountInfo()
    {
        base.AccountInfo();
        Console.WriteLine($"Тип: Бизнес, Компания: {CompanyName}, Сфера: {BusinessType}");
        Console.WriteLine($"Налоговый номер: {TaxId}, Ежемесячная комиссия: {MonthlyFee:C}");
        Console.WriteLine($"Овердрафт: {(HasOverdraft ? "Да" : "Нет")}, Лимит: {OverdraftLimit:C}");
        Console.WriteLine($"Баллы лояльности: {_loyaltyPoints}, Комиссия: {TransactionFee}%");
        Console.WriteLine($"Доступный баланс: {GetAvailableBalance():C}");
    }
}

public class AccountManager<T> where T : SavingAccount
{
    private List<T> accounts = new List<T>();
    private readonly INotificationService _notificationService;
    private readonly IAuditService _auditService;

    public AccountManager(INotificationService notificationService, IAuditService auditService)
    {
        _notificationService = notificationService;
        _auditService = auditService;
    }

    public void AddAccount(T account)
    {
        accounts.Add(account);
        _auditService.LogTransaction(account.AccountId, "ACCOUNT_CREATED", 0, DateTime.Now);
        _notificationService.SendNotification("Счет добавлен в менеджер", account.AccountId);
        Console.WriteLine($"Добавлен счет в менеджер: {account.AccountId}");
    }

    public T FindAccount(string accountId)
    {
        return accounts.Find(a => a.AccountId == accountId);
    }

    public List<T> GetAccountsByOwner(string ownerName)
    {
        return accounts.FindAll(a => a.OwnerName == ownerName);
    }

    public void ShowAllAccounts()
    {
        Console.WriteLine($"\nВсе счета в менеджере ({typeof(T).Name}):");
        foreach (var account in accounts)
        {
            account.AccountInfo();
        }
    }

    public void ApplyInterestToAll()
    {
        foreach (var account in accounts)
        {
            account.ApplyInterest();
        }
    }

    public void ApplyMonthlyFees()
    {
        foreach (var account in accounts)
        {
            if (account is AdvancedBusinessAccount businessAccount)
            {
                businessAccount.ApplyMonthlyFee();
            }
        }
    }
}

public class TransactionService<T> where T : SavingAccount
{
    private readonly IAuditService _auditService;

    public TransactionService(IAuditService auditService)
    {
        _auditService = auditService;
    }

    public bool Transfer(T fromAccount, T toAccount, decimal amount)
    {
        if (fromAccount.Withdraw(amount))
        {
            toAccount.Deposit(amount);
            _auditService.LogTransaction(fromAccount.AccountId, "TRANSFER_OUT", amount, DateTime.Now);
            _auditService.LogTransaction(toAccount.AccountId, "TRANSFER_IN", amount, DateTime.Now);
            return true;
        }
        return false;
    }
}

public class Bank
{
    private List<SavingAccount> accounts = new List<SavingAccount>();
    private readonly INotificationService _notificationService;
    private readonly IAuditService _auditService;

    public Bank(INotificationService notificationService, IAuditService auditService)
    {
        _notificationService = notificationService;
        _auditService = auditService;
    }

    public void AddAccount(SavingAccount account)
    {
        accounts.Add(account);
        _auditService.LogTransaction(account.AccountId, "ACCOUNT_ADDED", 0, DateTime.Now);
        Console.WriteLine($"Добавлен счет: {account.AccountId}");
    }

    public void Transfer(string fromId, string toId, decimal amount)
    {
        var fromAccount = accounts.Find(a => a.AccountId == fromId);
        var toAccount = accounts.Find(a => a.AccountId == toId);
        if (fromAccount != null && toAccount != null)
        {
            Console.WriteLine($"Перевод {amount} с {fromId} на {toId}:");
            if (fromAccount.Withdraw(amount))
            {
                toAccount.Deposit(amount);
                _auditService.LogTransaction(fromId, "TRANSFER_OUT", amount, DateTime.Now);
                _auditService.LogTransaction(toId, "TRANSFER_IN", amount, DateTime.Now);
            }
        }
        else
        {
            Console.WriteLine("Ошибка: счет не найден.");
        }
    }

    public void ShowAllAccounts()
    {
        Console.WriteLine("\nВсе счета:");
        foreach (var account in accounts)
        {
            account.AccountInfo(true);
        }
    }

    public void ApplyInterestToAll()
    {
        Console.WriteLine("\nНачисление процентов на все счета:");
        foreach (var account in accounts)
        {
            account.ApplyInterest();
        }
    }

    public void FreezeAccount(string accountId)
    {
        var account = accounts.Find(a => a.AccountId == accountId);
        account?.FreezeAccount();
    }

    public void UnfreezeAccount(string accountId)
    {
        var account = accounts.Find(a => a.AccountId == accountId);
        account?.UnfreezeAccount();
    }
}


var notificationService = new EmailNotificationService();
var auditService = new FileAuditService();

var bank = new Bank(notificationService, auditService);

SavingAccount account1 = new SavingAccount("001", 1000, 5, "Иван Иванов", notificationService, auditService);
AdvancedBusinessAccount businessAccount = new AdvancedBusinessAccount("505", 5000, 3, "ООО 'Ромашка'", "ООО 'Ромашка'", "Розничная торговля", "1234567890", notificationService, auditService);

account1.AccountUpdated += OnAccountUpdated;
static void OnAccountUpdated(string message)
{
    Console.WriteLine(message);
}
List<SavingAccount> accountsList = new List<SavingAccount>();
accountsList.Add(new SavingAccount("002", 1001, 6, "Петр Петров", notificationService, auditService));
accountsList.Add(new SavingAccount("003", 1002, 7, "Василий Васильев", notificationService, auditService));
accountsList.Add(new SavingAccount("004", 1003, 8, "Александра Александрова", notificationService, auditService));

foreach (var account in accountsList)
{
    account.DisplayInfo();
    Console.WriteLine();
}

bank.AddAccount(account1);
bank.AddAccount(businessAccount);

Console.WriteLine("Тестирование новых функций");
account1.Deposit(200, "Зарплата");
account1.ChangeCurrency("USD");
account1.AccountInfo(true);

Console.WriteLine("\n Бизнес счет");
businessAccount.Deposit(1000);
businessAccount.SetOverdraftLimit(10000);
businessAccount.AccountInfo(true);

Console.WriteLine("\n Явная реализация интерфейса");
ITransactionFee transactionFee = businessAccount;
ILoyaltyProgram loyaltyProgram = businessAccount;
        
Console.WriteLine($"Комиссия: {transactionFee.TransactionFee}%");
Console.WriteLine($"Баллы лояльности: {loyaltyProgram.LoyaltyPoints}");

Console.WriteLine("\n Переводы");
bank.Transfer("001", "505", 300);
bank.ShowAllAccounts();

Console.WriteLine("\n Начисление процентов");
bank.ApplyInterestToAll();

Console.WriteLine("\n Управление счетами");
bank.FreezeAccount("001");
bank.UnfreezeAccount("001");

DisplayDelegate displayAccount = account1.DisplayInfo;

displayAccount();